In [56]:
import os
import pandas as pd
from tqdm import tqdm
import ast
from datetime import datetime as dt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly
import igraph as ig
import plotly.graph_objs as go
import numpy as np

USERNAME = 'pattersongp'
API_KEY = 'VVZGgCcQVs2EkznPaIY3'
plotly.tools.set_credentials_file(username=USERNAME, api_key=API_KEY)

%matplotlib inline

In [57]:
df=pd.DataFrame.from_csv('./data/restricted/featured_data.csv')
df['poster'] = df['poster'].apply(lambda x: ast.literal_eval(x))
df['reactions'] = df['reactions'].apply(lambda x: ast.literal_eval(x))
df['comments'] = df['comments'].apply(lambda x: ast.literal_eval(x))

### Plotting activity over time

In [61]:
plot = df.copy()
plot = plot[['post_time','activity']]
plot['post_time'] = pd.to_datetime(plot['post_time'])
plot = plot.set_index('post_time')
x=plot.index
y=plot[plot['activity']>0]

# Plotly scatter plot
trace = go.Scattergl(
    x = x,
    y = y['activity'],
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)
layout= go.Layout (
    title="Post Activity for All Posts",
    xaxis = dict(
        title='Time'
    ),
    yaxis = dict(
        title='Activity'
    ),
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='activity_scatter_byPost')

#### Looks a bit dense, so group it by week to thin it out a bit

In [60]:
# Group by week
month_groups = plot.groupby(pd.TimeGrouper(freq='W')).count()

x=month_groups.index
y=month_groups['activity']

# Plotly scatter plot
trace = go.Scattergl(
    x = x,
    y = y,
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)
layout= go.Layout (
    title="Activity Grouped by Week",
    xaxis = dict(
        title='Time'
    ),
    yaxis = dict(
        title='Activity'
    ),
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='activity_scatter_byWeek')

In [54]:
import datetime

days = {
    0:"Sunday",
    1:"Monday",
    2:"Tuesday",
    3:"Wedneday",
    4:"Thursday",
    5:"Friday",
    6:"Saturday"
}
df_days=pd.DataFrame.from_dict(days,orient='index')
df_days['number']=df_days.index
df_days=df_days.set_index(0)

# Map the days of the week
def week_day(x):
    return days[datetime.datetime.strptime(x,'%Y-%m-%d').weekday()]

plot = df.copy()[['post_time','activity']]
plot['post_time'] = plot['post_time'].apply(lambda x: week_day(x))

# Group by day of the week
day_groups = plot.groupby(plot.post_time).count()
day_groups = df_days.join(day_groups)
day_groups.sort_values(by='number')

# Plot the chart, make it nice
x = day_groups.index
y = day_groups['activity']
trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    marker = dict(
        size = np.sqrt(day_groups['activity']),
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)
layout= go.Layout (
    title="Activity per Day",
    xaxis = dict(
        title='Day of Week'
    ),
    yaxis = dict(
        title='Activity'
    ),
)

data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='activity_scatter_byDayOfWeek')

In [53]:
emojis = pd.DataFrame.from_csv('./data/emoji_data.csv')
emojis = emojis.head(10)
# Plot the chart, make it nice
x = emojis.index
y = emojis['count']

trace = go.Scatter(
    x = x,
    y = y,
    mode = 'markers',
    marker = dict(
        size = emojis['count'],
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)

layout = go.Layout (
    title="Top Ten Emojis",
    hovermode=False,
    yaxis=dict(
        title='Usage'
    ),
    xaxis=dict(
        title='Emoji'
    ),
    showlegend=False
)

data = [trace]
figure = go.Figure(data=data, layout=layout)
py.iplot(figure,filename='emoji_scatter')

In [48]:
data = [
    go.Scatter(                         # all "scatter" attributes: https://plot.ly/python/reference/#scatter
        x=[1, 2, 3],                    # more about "x":  /python/reference/#scatter-x
        y=[3, 1, 6],                    # more about "y":  /python/reference/#scatter-y
        marker=dict(                    # marker is an dict, marker keys: /python/reference/#scatter-marker
            color="rgb(16, 32, 77)"     # more about marker's "color": /python/reference/#scatter-marker-color
        )
    ),
    go.Bar(                         # all "bar" chart attributes: /python/reference/#bar
        x=[1, 2, 3],                # more about "x": /python/reference/#bar-x
        y=[3, 1, 6],                # /python/reference/#bar-y
        name="bar chart example"    # /python/reference/#bar-name
    )
]

layout = go.Layout(             # all "layout" attributes: /python/reference/#layout
    title="simple example",     # more about "layout's" "title": /python/reference/#layout-title
    xaxis=dict(                 # all "layout's" "xaxis" attributes: /python/reference/#layout-xaxis
        title="time"            # more about "layout's" "xaxis's" "title": /python/reference/#layout-xaxis-title
    )
)

figure = go.Figure(data=data, layout=layout)

py.iplot(figure, filename='reference-graph')